In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
from pprint import pprint

# Importing the models to be tested
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.covariance import EmpiricalCovariance
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from auxiliar_func import *
from plot_func import *

## Parameter tunning: cross-validation

In [2]:
target = 'income_50k'               # target variable
df_tr = pd.read_csv('../train.csv') # training set

TARGET_METRIC = 'f1_macro'          # metric to be used in the grid search
SEED = 42                           # seed for reproducibility

# Grid of preprocessing hyperparameters for each model
prep_params_grid = {
    'scaling': [None, 'minmax', 'standard'],
    'imputation': ['mode'],
    'cat_age': [False, True],
    'merge_capital': [False, True],
    'downsampling_method': ['random'],
    'target_freq': [0.75, 0.8, 0.85, 0.9],
    'generate_dummies': [True]
}

def n_comb(grid: dict, print_=True):
    """Returns the number of combinations to be tested given a grid of parameters"""
    n = 1
    for k in grid.keys():
        n *= len(grid[k])
    if print_:
        print(f'Number of model combinations to be tested: {n}')
    else:
        return n

def test_model(mod, prep_grid, mod_grid, name, rewrite=False, **kwargs):
    """Tests a model with all the possible combinations of preprocessing parameters and hyperparameters"""
    if rewrite or not os.path.exists(f'./results/results_{name}.csv'):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore") # to avoid convergence warnings
                results = search_best_combination(mod, mod_grid, prep_grid, df_tr, target_metric=TARGET_METRIC, cv=5, N=15, **kwargs)
                results.to_csv(f'./results/results_{name}.csv', index=False)
        except Exception as e:
            print(e)
            return None

n_comb(prep_params_grid)

Number of model combinations to be tested: 48


### Naive Bayes

In [3]:
mod = GaussianNB()

mod_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

n_comb(mod_grid)

test_model(mod, prep_params_grid, mod_grid, 'nb', rewrite=True)

Number of model combinations to be tested: 5
===Iteration 1===
Searching preprocessing parameters...


### Linear Discriminant Analysis (LDA)

In [ ]:
mod_par_grid = {
    'solver': ['svd'], # 'svd' is faster and recommended for large datasets
    'priors': [None], # By default, the class proportions are inferred from the training data.
    'tol': [1e-4, 1e-3, 1e-2], # Absolute threshold for a singular value of X to be considered significant, used to estimate the rank of X.
}

n_comb(mod_par_grid) 

lda = LDA()
test_model(lda, prep_params_grid, mod_par_grid, 'lda')

Number of model combinations to be tested: 3


### Quadratic Discriminant Analysis (QDA)

In [ ]:
mod_par_grid = {
    'priors': [None], # By default, the class proportions are inferred from the training data.
    'reg_param': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], # Regularizes the per-class covariance estimates: S = (1 - reg_param) * S + reg_param * np.eye(n_features),
    'store_covariance': [True, False], # If True, the covariance matrices are computed and stored in the self.covariance_ attribute.
    'tol': [1e-4, 1e-3, 1e-2], # Absolute threshold for a singular value of X to be considered significant, used to estimate the rank of X. Does not affect the predictions.
}

n_comb(mod_par_grid)

qda = QDA()
test_model(qda, prep_params_grid, mod_par_grid, 'qda')

Number of model combinations to be tested: 36


### K-Nearest Neighbors (KNN)

In [ ]:
mod_par_grid = {
    'n_neighbors': [7, 9, 11, 13, 15, 17, 19], # Number of neighbors to use by default for kneighbors queries.
    'weights': ['uniform', 'distance'], # weight function used in prediciton
}

n_comb(mod_par_grid)

knn = KNN()
test_model(knn, prep_params_grid, mod_par_grid, 'knn')

Number of model combinations to be tested: 14


### Logistic regression

In [ ]:
mod_par_grid = {
    'penalty': ['l1', 'l2'], # Used to specify the norm used in the penalization.
    'C': [0.1, 1, 10], # Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
    'class_weight': [None, 'balanced'], 
    'fit_intercept': [True, False], # Specifies if a constant should be added to the decision function.
    'intercept_scaling': [0.1, 1, 10],  
    'max_iter': [1000], # Maximum number of iterations taken for the solvers to converge.
    'multi_class': ['auto'],  
    'random_state': [SEED],
    'solver': ['saga'] # Algorithm to use in the optimization problem.
}

n_comb(mod_par_grid)

logreg = LogisticRegression()
test_model(logreg, prep_params_grid, mod_par_grid, 'logreg')

Number of model combinations to be tested: 72


### Support Vector Machines (SVM)

In [ ]:
mod_par_grid = {
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge','hinge'],
    'dual': [False],
    'C': [0.1, 1, 10],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'random_state': [SEED]
}

n_comb(mod_par_grid)

svm = LinearSVC()
test_model(svm, prep_params_grid, mod_par_grid, 'svm')

Number of model combinations to be tested: 96


### Random forest

In [ ]:
mod_par_grid = {
    'n_estimators': [50, 65, 75, 100],
    'criterion': ['gini'],
    'max_depth': [None, 25, 30, 35, 40],
    'max_features': ['sqrt', 'log2'],
    'random_state': [SEED],
    'verbose': [0],
    'warm_start': [False],
    'class_weight': [None, 'balanced']
}

n_comb(mod_par_grid)

rf = RandomForestClassifier()
test_model(rf, prep_params_grid, mod_par_grid, 'rf')

Number of model combinations to be tested: 80


### Gradient Boosting (XGBoost)

In [ ]:
mod_par_grid = {
    'n_estimators': [50, 65, 75],
    'max_depth': [None, 25, 30, 35],
    'learning_rate': [0.1, 0.2, 0.3],
    'booster': ['gbtree', 'gblinear'],
}

n_comb(mod_par_grid)

# for all categorical features, we need to remove special characters like '[', ']', and '<'
df_tr = df_tr.applymap(lambda x: x.replace('[', '').replace(']', '').replace('<', '') if isinstance(x, str) else x)

xgb = XGBClassifier()
test_model(xgb, prep_params_grid, mod_par_grid, 'xgb')

Number of model combinations to be tested: 72
===Iteration 1===
Searching preprocessing parameters...
it: 48/48
Searching model parameters...
it: 72/72
Best metric: 0.7904680227255335
Best preprocessing parameters: [{'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': True, 'downsampling_method': 'random', 'target_freq': 0.8, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.8, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': True, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': True, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': None, 'imputation

### Gradient Boosting

In [ ]:
prep_params_grid3 = {
    'scaling': [None],
    'imputation': ['mode'],
    'cat_age': [False],
    'merge_capital': [False, True],
    'downsampling_method': ['random'],
    'target_freq': [0.8, 0.85, 0.9],
    'generate_dummies': [False]
}

# a first preprocess to get the categorical features
df_tr_pre = preprocessing(df_tr, imputation='mode', cat_age=False, generate_dummies=False)
X_train, y_train = df_tr_pre.drop(target, axis=1), df_tr_pre[target]
cat_features = list(X_train.select_dtypes(include=['category']).columns)

mod_par_grid = {
    'iterations': [500, 750],
    'depth': [1, 2, 4, 6],
    'border_count': [32, 64, 96],
    'random_seed': [SEED],
    'verbose': [0],
    'loss_function': ['Logloss'],
    'eval_metric': ['F1', 'AUC'],
    'cat_features': [cat_features],
}

n_comb(mod_par_grid)

cat_model = CatBoostClassifier()
test_model(cat_model, prep_params_grid3, mod_par_grid, 'catboost')

Number of model combinations to be tested: 48
